In [1]:
#import necessary pacakages
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re
import string    


[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [2]:
#loading the data as DataFrame
#reading and storing Excel file into variabe Data
data=pd.read_excel(r'C:\Users\USER\Desktop\Input.xlsx')
df=data

In [3]:
df

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [4]:

# loop through each row in the df
for index, row in df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']

    # make a request to url
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}  # giving user access
    try:
        response = requests.get(url, headers=headers)  # loading text in url
    except:
        print("can't get response of {}".format(url_id))
        continue

    # create a beautifulsoup object
    try:
        soup = BeautifulSoup(response.content, 'html.parser')  # parsing url text
    except:
        print("can't get page of {}".format(url_id))
        continue

    # find text
    try:
        content = soup.findAll(attrs={'class': 'td-post-content'})  # extracting only text part
        content = content[0].text.replace('\xa0', "  ").replace('\n', "  ")  # replace end line symbol with space
    except:
        print("can't get text of {}".format(url_id))
        continue

    # find title
    try:
        title = soup.find('h1').get_text() # extracting title of website
        #title = title[16].text.replace('\n', "  ").replace('/', "")
    except:
        print("can't get title of {}".format(url_id))
        continue

    text = title + '.' + content  # merging title and content text
    text = text.replace('\n', "  ").replace('/', "")

    # write title and text to the file
    file_name = 'C:/Users/USER/Desktop/notes2/' + str(url_id) + '.txt'
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(text)

    # create pandas Series from text
    df1 = pd.Series([text])
    # do something with df1, e.g. concatenate it with another Series or DataFrame




can't get text of 44
can't get text of 57
can't get text of 144


In [5]:
# Directories
text_dir = 'C:/Users/USER/Desktop/notes2/'
stopwords_dir = "C:/Users/USER/Desktop/StopWords"
sentiment_dir = "C:/Users/USER/Desktop/MasterDictionary"

In [6]:
# Load stop words into a set called "stop_words"
stop_words = set()

# Loop over all files in the stopwords directory
for file in os.listdir(stopwords_dir):
    # Open the file and read its contents
    with open(os.path.join(stopwords_dir, file), 'r', encoding='ISO-8859-1') as f:
        file_contents = f.read()
        # Split the file contents by line and add to the set of stop words
        stop_words.update(file_contents.splitlines())


# load all text files from the directory into a list called "docs"
docs = []
for text_file in os.listdir(text_dir):
    with open(os.path.join(text_dir, text_file), 'r', encoding='ISO-8859-1') as f:
        text = f.read()
        # Tokenize the text and remove stop words
        filtered_text = [word.lower() for word in word_tokenize(text) if word.lower() not in stop_words]
        # Add filtered text to the list of documents
        docs.append(filtered_text)


In [7]:
pos = set()
neg = set()

for file in os.listdir(sentiment_dir):
    with open(os.path.join(sentiment_dir, file), 'r', encoding='ISO-8859-1') as f:
        if file == 'positive-words.txt':
            pos.update(f.read().splitlines())
        else:
            neg.update(f.read().splitlines())

In [8]:
# now collect the positive  and negative words from each file
# calculate the scores from the positive and negative words 
positive_words = []
Negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

In [9]:

#iterate through the list of docs
for i in range(len(docs)):
    positive_words.append([word for word in docs[i] if word.lower() in pos])
    Negative_words.append([word for word in docs[i] if word.lower() in neg])
    positive_score.append(len(positive_words[i]))
    negative_score.append(len(Negative_words[i]))
    polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
    subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))

In [10]:

# Average Sentence Length = the number of words / the number of sentences
# Percentage of Complex words = the number of complex words / the number of words 
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

avg_sentence_length = []
percentage_of_complex_words  =  []
fog_index = []
complex_word_count =  []
avg_syllable_word_count =[]


In [11]:
import os
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def measure(file):
    with open(os.path.join(text_dir, file), 'r', encoding='utf-8') as f:
        text = f.read()

    # Remove punctuations
    text = re.sub(r'[^\w\s.]', '', text)

    # Split the text into sentences
    sentences = text.split('.')

    # Total number of sentences in the text file
    num_sentences = len(sentences)

    # Total words in the text file
    words = [word for word in text.split() if word.lower() not in stop_words]
    num_words = len(words)

    # Complex words have more than 2 syllables
    complex_words = [word for word in words if sum(1 for letter in word.lower() if letter in 'aeiou') > 2]

    # Count syllables in each word
    syllable_count = 0
    syllable_words = []
    for word in words:
        if word.endswith('es'):
            word = word[:-2]
        elif word.endswith('ed'):
            word = word[:-2]
        syllable_count_word = sum(1 for letter in word.lower() if letter in 'aeiou')
        if syllable_count_word >= 1:
            syllable_words.append(word)
            syllable_count += syllable_count_word

    # Compute metrics
    avg_sentence_length = num_words / num_sentences
    avg_syllable_word_count = syllable_count / len(syllable_words)
    percentage_complex_words = len(complex_words) / num_words
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    return avg_sentence_length, percentage_complex_words, fog_index, len(complex_words), avg_syllable_word_count


In [12]:
# Iterate through each file in the directory
for file in os.listdir(text_dir):
    x, y, z, a, b = measure(file)
    avg_sentence_length.append(x)
    percentage_of_complex_words.append(y)
    fog_index.append(z)
    complex_word_count.append(a)
    avg_syllable_word_count.append(b)

In [13]:
def cleaned_words(file):
    with open(os.path.join(text_dir, file), 'r',encoding='utf-8') as f:
        text = f.read()
        text = re.sub(r'[^\w\s]', '', text)
        words = [word for word in text.split() if word.lower() not in stop_words]
        length = sum(len(word) for word in words)
        average_word_length = length / len(words)
        return len(words), average_word_length

word_count = []
average_word_length = []

for file in os.listdir(text_dir):
    x, y = cleaned_words(file)
    word_count.append(x)
    average_word_length.append(y)


In [14]:
# To calculate Personal Pronouns mentioned in the text, we use regex to find 
# the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken
#  so that the country name US is not included in the list.
def count_personal_pronouns(file):
    with open(os.path.join(text_dir,file), 'r',encoding='utf-8') as f:
        text = f.read()
        personal_pronouns = ["I", "we", "my", "ours", "us"]
        count = 0
    for pronoun in personal_pronouns:
        count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
    return count

pp_count = []
for file in os.listdir(text_dir):
    x = count_personal_pronouns(file)
    pp_count.append(x)

In [15]:
output_df = pd.read_excel(r'C:\Users\USER\Desktop\Output Data Structure.xlsx')


In [16]:
# URL_ID 44 ,57, 144 does not exists i,e. page does not exist, throughs 404 error
# so we are going to drop these rows from the table
output_df = output_df.drop(output_df[output_df['URL_ID'].isin([44, 57, 144])].index)

In [17]:
# These are the required parameters 
variables = [positive_score,
            negative_score,
            polarity_score,
            subjectivity_score,
            avg_sentence_length,
            percentage_of_complex_words,
            fog_index,
            avg_sentence_length,
            complex_word_count,
            word_count,
            avg_syllable_word_count,
            pp_count,
            average_word_length]

In [18]:
# write the values to the dataframe
for i, var in enumerate(variables):
    if len(var) != len(output_df):
        print(f"Error: length of {var} does not match length of dataframe column")
    else:
        output_df.iloc[:,i+2] = var

In [19]:
#now save the dataframe to the disk
output_df.to_csv('Output_Data2.csv',index=False)